In [1]:
!curl https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -O

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1089k  100 1089k    0     0  2413k      0 --:--:-- --:--:-- --:--:-- 2431k


In [2]:
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [3]:
print(len(text))
print(text[:1000])

1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for re

In [4]:
vocabulary = sorted(list(set(text)))
vocab_size = len(vocabulary)
print("".join(vocabulary))
print(vocab_size)

# declare a list containing all the chracters in the source text (65 characters including a space)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
# Tokenizing
# map each character into a corresponding number

encode_mapping = {ch: i for i, ch in enumerate(vocabulary)}
decode_mapping = {i: ch for i, ch in enumerate(vocabulary)}

encode = lambda s: [encode_mapping[c] for c in s]
decode = lambda l: "".join([decode_mapping[i] for i in l])

In [6]:
print(encode("hello world"))
print(decode(encode("hello world")))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


토큰화하는 방식은 다양하게 있을 수 있음

e.g., OpenAI의 tiktoken (BPE), Google의 sentence-piece


In [7]:
import tiktoken

In [8]:
encoding = tiktoken.get_encoding("gpt2")

In [9]:
print(type(encoding))
encoding.n_vocab

<class 'tiktoken.core.Encoding'>


50257

In [10]:
vocab = {token_id: encoding.decode([token_id]) for token_id in range(encoding.n_vocab)}

In [11]:
for i in range(10):
    print(f"{i}: {repr(vocab[i])}")

0: '!'
1: '"'
2: '#'
3: '$'
4: '%'
5: '&'
6: "'"
7: '('
8: ')'
9: '*'


In [19]:
encoding.encode("annyeong world")

[1236, 5948, 506, 995]

In [13]:
import torch

encoded = torch.tensor(encode(text), dtype=torch.long)

c:\Users\lych1\OneDrive\Desktop\project-my-gpt\.mygpt\lib\site-packages\torch\_subclasses\functional_tensor.py:275: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [14]:
encoded.shape, encoded.dtype
print(encoded[:100])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [15]:
# divide the source text into a train set and a validation set
# this will be useful to check if the model is over/underfitting in the future

n = int(0.9 * len(encoded))

train_set = encoded[:n]
validation_set = encoded[n:]

In [ ]:
# since training the texts all at once demands significant amount of computation,
# the text should be divided into small chunks
# Having small chunks not only help with the efficiency,
# but also helps model adapt to different context sizes.
# For example, to infer 57, not only considering the context size of 1, which is 56
# the model can also consider [47, 56] as well as [18, 47, 56]

chunk_size = 8
train_set[
    : chunk_size + 1
]  # 1 should be added because the character after a whole chunk should be trained too

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In the context of 18, 47 is likely to come next

In the context of 18 and 47, 56 is likely to come next and so on


In [ ]:
x = train_set[:chunk_size]  # example input train data
y = train_set[1 : chunk_size + 1]  # the target character to be predicted
for t in range(chunk_size):
    context = x[: t + 1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [22]:
print(x)
print(y)

tensor([18, 47, 56, 57, 58,  1, 15, 47])
tensor([47, 56, 57, 58,  1, 15, 47, 58])


In [ ]:
# Mini batches stacked up in a single tensor for GPU efficiency

torch.manual_seed(42)

batch_size = 4
block_size = 8

def get_batch(split):
    data = train_set if split == "train" else validation_set
    ix = torch.randint(len[data - block_size, (batch_size,)])
    x = torch.stack[data[i:i+block_size] for i in ix]
    y = torch.stack[data[i+1:i+block_size+1] for i in ix]
    return x, y